In [47]:
import pandas as pd
import numpy as np
import json

In [58]:
df_searches = pd.read_excel("searches_candidates_selection_rank.xlsx")
df_searches = df_searches[df_searches['query'] != 'undefined']

In [69]:
def slug_only(dr):
    candidates = json.loads(dr.replace("'", '"'))    
    try :
        return json.dumps([c['slug'] for c in candidates])
    except TypeError:
        print('ERROR:\n' + dr)
        return None

df_searches['docs_slug'] = df_searches.docs_res.apply(lambda dr : slug_only(dr))

ERROR:
['fiches_ministere_travail/le-registre-unique-du-personnel#Quelle-forme-doit-revetir-le-registre-unique-du-personnel', 'fiches_service_public/registres-du-personnel-obligatoires', 'fiches_ministere_travail/le-registre-unique-du-personnel#Les-stagiaires-sont-ils-concernes', 'fiches_ministere_travail/le-registre-unique-du-personnel#Quelles-sont-les-sanctions-en-cas-d-infraction', 'fiches_ministere_travail/le-registre-unique-du-personnel#Quels-sont-les-employeurs-concernes', 'fiches_service_public/lanceur-dalerte-dans-lentreprise-quelles-sont-les-regles', 'fiches_ministere_travail/le-registre-unique-du-personnel#Pour-quels-salaries']


# Evaluation : group queries and get rank for each candidates 
Objective : compute NDCG / MAP / MRR

In [269]:
# most frequent queries with hit
all_ranks = []

threshold = 7
for g, rows in df_searches[df_searches['rank'] >= 0].groupby(["docs_slug"]):
    n = rows.shape[0]
    q = set(rows['query'].tolist())
    algo = json.loads(rows.docs_res.tolist()[0].replace("'",'"'))[0]['algo']
    preq = 'pre-qualified' in algo
    candidates = g
    if (n > threshold):        
        print(f'{q} -> {n}')
        print(algo)
        ranks = rows['rank'].value_counts().sort_index()
        candidates_array = json.loads(candidates)
        j = len(candidates_array)
        complete_ranks = [0] * j
        ids = []
        for i in range(0, j):
            if i in ranks.index:
                complete_ranks[i] = ranks[i]             
            ids.append({'slug':candidates_array[i], 'count':complete_ranks[i], 'queries':q, 'preq':preq})
        all_ranks.append(ids)

{"je suis à la retraite dpuis le 1er janvier 2020 et je n'est reçu aucune indemnition cela fait 29 ans que je travaille dans la mème entreprise"} -> 17
semantic
{'indemnité de départ à la retraite', 'indemnité départ à la retraite', 'PRIME DE DEPART A LA RETRAITE', 'prime de départ à la retraite', 'indemnité de départ en retraite', 'indemnité départ retraite', 'quel est le montant de la prime de départ à la retraite ?', 'comment calculer les indemnités de départ à la retraite ?'} -> 80
pre-qualified
{'indemnités'} -> 15
semantic
{"maladie pendant période d'essai", "prolongation période d'essai accident du travail", "arrêt maladie pendant période d'essai"} -> 19
pre-qualified
{'renouvellement cdd', 'renouveler CDD', 'RENOUVELLEMENT CDD', 'renouveler cdd', 'prolongation cdd'} -> 28
pre-qualified
{'Ancienneté entreprise', 'ANCIENNETE ET CONGES PARENTAL', 'ancienneté et congé sabbatique', 'calcul ancienneté professionnelle', 'ancienneté et reprise par un nouvel employeur', 'ancienneté calc

{'bruit'} -> 10
both
{'droit de retrait chaleur', 'quelle température pour travailler dans un bureau ?', 'températures extrêmes code du travail', 'température légale pour travailler', 'chaleur et canicule au travail'} -> 26
pre-qualified
{'TRAVAIL EN HAUTEUR', 'comment prévenir les risques de chutes de hauteur ?', 'travaux sur escabeaux', 'coordonnateur en sécurité et protection de la santé et risque de chute de hauteur', 'travail sur corde', 'risque de chute de hauteur', 'Harnais', 'risques liés aux chutes de hauteur', 'protection collective contre les chute de hauteur', 'travail sur échafaudage', 'garde-corps', 'prévention des risques de chute de hauteur', 'garde corps', 'échafaudage', 'travail en hauteur', 'protection individuelle contre les risques de chute de hauteur', 'plan de prévention et risques de chute de hauteur'} -> 44
pre-qualified
{'pénibilité comment la gerer dans les entreprise'} -> 11
semantic
{'rupture contrat d apprentissage', "rupture contrat d'apprentissage", 'je 

KeyError: 'algo'

In [270]:
print(len(all_ranks))
#print()
#print(sorted(all_ranks[6], key=lambda x: x['count'], reverse=True))

68


In [111]:
import requests

params = {
    "y_pred" : ["a", "b", "c", "w", "k","e"], # y_pred (array): list of documents id predicted by the system
    "y_true" : ["a", "b", "c", "e"], # y_true (array): documents id scored by humans sorted from most relevant to least relevant
    "y_score" : {
        "a":5,
        "b":3,
        "c":2,
        "e":2
            }, # y_score is a dict {"doc_id":"score"} of documents assigned as relevant y humans with the associated scores (ordered internally)
    "method" : "all", # one of ["precision", "recall", "dcg", "mrr", "all"]
    "k": 3 # k integer preferaly =< to length(y_pred)
}



{'dcg': 1.0, 'mrr': 1.0, 'precision': 1.0, 'recall': 0.75}

In [271]:
def sort_ranks(ranks):
    return sorted(ranks, key=lambda x: x['count'], reverse=True)

def compute_metrics(i, verbose=False):
    scores = dict([(r['slug'], int(r['count'])) for r in all_ranks[i]])
    params = {

        "y_pred": [c['slug'] for c in all_ranks[i]],
        #"y_pred": [c['slug'] for c in sort_ranks(all_ranks[i]) if c['count'] > 0],
        "y_true" : [c['slug'] for c in sort_ranks(all_ranks[i]) if c['count'] > 0],
        "y_score" : scores,
        "method" : "all",
        "k":5
    }
    r = requests.post("http://localhost:4545/api/score", json = params)
    
    if r.status_code != 200:
        print("Error with at index " + str(i))
        return {'dcg': 1.0, 'mrr': 1.0, 'precision': 1.0, 'recall': 1.0}

    
    result = r.json()
    if verbose:
        print(i)
        print(all_ranks[i][0]['queries'])
        print(json.dumps(scores, indent=2))
        print(result)
    
    return result

In [272]:
# index of ranks with pre-qualified
# index of ranks without

preq_indices = [i for i in range(0, len(all_ranks)) if all_ranks[i][0]['preq']]
other_indices = [i for i in range(0, len(all_ranks)) if not all_ranks[i][0]['preq']]

In [273]:
len(preq_indices)

37

In [274]:
preq_scores = [compute_metrics(i) for i in preq_indices]
other_scores = [compute_metrics(i) for i in other_indices]

In [275]:
pd.DataFrame(preq_scores)['dcg'].describe()

count    37.000000
mean      0.969830
std       0.067478
min       0.682678
25%       0.975100
50%       0.996734
75%       1.000000
max       1.000000
Name: dcg, dtype: float64

In [276]:
pd.DataFrame(other_scores)['dcg'].describe()

count    31.000000
mean      0.776930
std       0.207710
min       0.384453
25%       0.595286
50%       0.810177
75%       0.965174
max       1.000000
Name: dcg, dtype: float64

In [286]:
all_ranks[preq_indices[30]]

[{'slug': '/fiche-ministere-travail/amenagement-du-temps-de-travail#Comment-mettre-en-place-un-dispositif-d-amenagement-du-temps-de-travail-nbsp',
  'count': 18,
  'queries': {'TEMPS DE TRAVAIL',
   'Temps de travail',
   'calcul temps de travail annuel',
   'durée du travail et temps de travail',
   'temps de travail',
   'temps de travail et aménagement du temps de travail',
   'temps de travail et durée légale',
   'temps de travail et heures supplémentaires',
   'temps de travail et temps de soirée',
   'temps de travail et travail à temps partiel',
   'temps de travail et travail à temps plein',
   'temps de travail minimum par jour'},
  'preq': True},
 {'slug': '/fiche-ministere-travail/la-duree-legale-du-travail#Quels-sont-les-temps-de-travail-comptabilises-dans-la-duree-legale-nbsp',
  'count': 71,
  'queries': {'TEMPS DE TRAVAIL',
   'Temps de travail',
   'calcul temps de travail annuel',
   'durée du travail et temps de travail',
   'temps de travail',
   'temps de travail e

In [284]:
[(i, preq_scores[i]) for i in range(0, len(preq_scores))]

[(0, {'dcg': 0.9989298154377032, 'mrr': 1.0, 'precision': 1.0, 'recall': 1.0}),
 (1, {'dcg': 1.0, 'mrr': 1.0, 'precision': 0.5, 'recall': 1.0}),
 (2, {'dcg': 0.9659167675637428, 'mrr': 1.0, 'precision': 1.0, 'recall': 1.0}),
 (3, {'dcg': 0.9505230792535972, 'mrr': 1.0, 'precision': 0.6, 'recall': 0.6}),
 (4, {'dcg': 0.9965893565378234, 'mrr': 1.0, 'precision': 1.0, 'recall': 1.0}),
 (5, {'dcg': 0.9978197507915574, 'mrr': 1.0, 'precision': 0.8, 'recall': 1.0}),
 (6,
  {'dcg': 0.9458802237582187,
   'mrr': 1.0,
   'precision': 0.4,
   'recall': 0.6666666666666666}),
 (7, {'dcg': 1.0, 'mrr': 1.0, 'precision': 1.0, 'recall': 1.0}),
 (8,
  {'dcg': 0.9984087397290985,
   'mrr': 1.0,
   'precision': 1.0,
   'recall': 0.5555555555555556}),
 (9, {'dcg': 1.0, 'mrr': 1.0, 'precision': 1.0, 'recall': 1.0}),
 (10,
  {'dcg': 0.9853269231034409, 'mrr': 1.0, 'precision': 1.0, 'recall': 1.0}),
 (11, {'dcg': 1.0, 'mrr': 1.0, 'precision': 0.6, 'recall': 1.0}),
 (12,
  {'dcg': 0.7417634207220976, 'mrr': 1

int